In [1]:
import warnings
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Dense, LSTM, Conv2D, Flatten, Rescaling, MaxPooling2D, BatchNormalization,ReLU
from sklearn.model_selection import train_test_split as tt
warnings.filterwarnings('ignore')

In [5]:
# df_train = pd.read_csv("train.csv")
# df_test = pd.read_csv("test.csv")
# df_sub = pd.read_csv("sample_submission.csv")

In [ ]:
x = np.asarray(df_train.drop('label',axis=1))
y = df_train['label']

x_train,x_val,y_train,y_val = tt(x,y,train_size=0.8,random_state=0)

In [ ]:
x_train = x_train.reshape(-1, 28, 28,1).astype("float32") / 255.0
x_val = x_val.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [ ]:
model = Sequential([
  tf.keras.Input(shape=(28,28,1)),
  Conv2D(16, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10,activation='softmax')
])

model.summary()
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.003),metrics=['accuracy'])


In [ ]:
epochs = 15
history = model.fit(x_train,y_train,batch_size=64,epochs=epochs,validation_data=(x_val, y_val),verbose=2)

In [ ]:
Visualizing the performance of the model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("digits_recogniser_model.h5")
del model

In [ ]:
# model = load_model('../cnn_py_files/digits_recogniser_model.h5')
# # Test Data
# x_test = np.asarray(df_test)
# x_test = x_test.astype("float32") / 255.0
# print(x_test[0].shape)
# def predictions():
#     preds = []
#     for i in range(x_test.shape[0]):
#         ip = x_test[i].reshape(-1, 28, 28, 1)
#         pred = model.predict(ip)
#         # print(pred)
#         # # Softmax - func
#         p = np.exp(pred)
#         s = p / p.sum()
#         preds.append(np.argmax(s))
#     return preds
#
# op = predictions()
# print(op)
#
#
# with open('op_file', 'wb') as f:
#     pickle.dump(op, f)


with open('op_file', 'rb') as fp:
    output = pickle.load(fp)

output = list(output)
index = list(df_sub.ImageId)
# print(index)
# output = np.asarray(output).reshape((-1,1))
# index = np.asarray(df_sub.ImageId).reshape((-1,1))
# print(output.shape)
# print(len(output))
new_df = pd.DataFrame({'ImageId':index,'Predicted_Label':output})
# new_df = new_df.set_index('ImageId')
print(new_df.head())

# Save to csv
new_df.to_csv('prediction_submission.csv',index=False)
